# Capstone: Musical Recommender

Kelly Slatery | US-DSI-10

In [1]:
# Imports 
import numpy as np
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import regex as re
from nltk.tokenize import RegexpTokenizer
import unicodedata

In [2]:
# Set view options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Import and Clean Data

In [3]:
# Import list of musicals
names = pd.read_csv('./data/musical_names.csv')
names.shape

(196, 1)

In [4]:
names.head()

,musical
0,Les Misérables
1,The Phantom of the Opera
2,Hamilton
3,West Side Story
4,Wicked


In [5]:
# Convert accented characters to ascii (normal) characters
names['musical'] = [unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8') for name in names['musical']]
names.head()

,musical
0,Les Miserables
1,The Phantom of the Opera
2,Hamilton
3,West Side Story
4,Wicked


In [6]:
# Remove all spaces and capital letters
names['musical'] = [re.sub('[^a-zA-Z0-9]', '', name).replace("'", '').lower() for name in names['musical']]
names.head()

,musical
0,lesmiserables
1,thephantomoftheopera
2,hamilton
3,westsidestory
4,wicked


# Data Collection

In [7]:
# Set up base url
baseurl1 = 'https://www.allmusicals.com/lyrics/'
baseurl2 = '/synopsis.htm'

# f'https://www.allmusicals.com/lyrics/{}/synopsis.htm'

In [8]:
# Set up empty list for musical synopses from allmusicals.com
synopses = []
missed_musicals = []

# Loop through all musicals in the list
for name in names['musical']:
    
    # Handle titles starting with "The"
    if name[:3] == 'the':
        
        # Set up list of links to loop over
        possible_names = [name[3:], name[3:] + 'the', name, name + 'the']
        i = 0        
        url = baseurl1 + possible_names[i] + baseurl2
        res = requests.get(url)

        # Check for which version of the musical name returns a result, if any
        while res.status_code != 200:
            i += 1
            try:
                url = baseurl1 + possible_names[i] + baseurl2
            except IndexError:
                print(f'Synopsis for "{name}" not listed on allmusicals.com. Try another source like Wikipedia.')
                url = ''
                missed_musicals.append(name)
                break
            res = requests.get(url)
    
    # Access webpage for all musicals not containing "The"
    else: 
        url = baseurl1 + name + baseurl2
        res = requests.get(url)

    # Get synposis
    if res.status_code == 200:
        soup = BeautifulSoup(res.content)
        synopsis_html = soup.find('div', {'id': 'page'}).text.replace('\r', '')
        synopsis_list = [words.strip() for words in synopsis_html.split('\n')[2:-2]]
        synopsis = ' '.join(synopsis_list).strip()
        synopses.append(synopsis)
        print(f'Synopsis for "{name}" added to list. Moving on to the next musical...')
    else:
        synopses.append('')
        missed_musicals.append(name)
        print(f'Synopsis for "{name}" not listed on allmusicals.com. Try another source like Wikipedia.')
        continue

Synopsis for "lesmiserables" added to list. Moving on to the next musical...
Synopsis for "thephantomoftheopera" added to list. Moving on to the next musical...
Synopsis for "hamilton" added to list. Moving on to the next musical...
Synopsis for "westsidestory" added to list. Moving on to the next musical...
Synopsis for "wicked" added to list. Moving on to the next musical...
Synopsis for "chicago" added to list. Moving on to the next musical...
Synopsis for "rent" added to list. Moving on to the next musical...
Synopsis for "thelionking" added to list. Moving on to the next musical...
Synopsis for "thebookofmormon" added to list. Moving on to the next musical...
Synopsis for "sweeneytoddthedemonbarberoffleetstreet" added to list. Moving on to the next musical...
Synopsis for "thesoundofmusic" added to list. Moving on to the next musical...
Synopsis for "fiddlerontheroof" added to list. Moving on to the next musical...
Synopsis for "intothewoods" added to list. Moving on to the next m

Synopsis for "thefantasticks" added to list. Moving on to the next musical...
Synopsis for "once" not listed on allmusicals.com. Try another source like Wikipedia.
Synopsis for "jekyllandhyde" not listed on allmusicals.com. Try another source like Wikipedia.
Synopsis for "beautifulthecarolekingmusical" not listed on allmusicals.com. Try another source like Wikipedia.
Synopsis for "brigadoon" added to list. Moving on to the next musical...
Synopsis for "onthetown" added to list. Moving on to the next musical...
Synopsis for "thewhostommy" added to list. Moving on to the next musical...
Synopsis for "bemorechill" added to list. Moving on to the next musical...
Synopsis for "thewiz" added to list. Moving on to the next musical...
Synopsis for "funhome" added to list. Moving on to the next musical...
Synopsis for "catchmeifyoucan" added to list. Moving on to the next musical...
Synopsis for "sweetcharity" added to list. Moving on to the next musical...
Synopsis for "mame" added to list. Mo

In [9]:
# Check that length of synopses is 196
print(len(synopses))

# Look at which musicals are missing synopses
print(missed_musicals)

196
['achorusline', 'grease', 'beautyandthebeast', 'gypsyamusicalfable', 'cinderella', 'billyelliotthemusical', 'schoolofrockthemusical', 'alittlenightmusic', 'porgyandbess', 'afunnythinghappenedonthewaytotheforum', 'agentlemansguidetoloveandmurder', 'once', 'jekyllandhyde', 'beautifulthecarolekingmusical', 'sisteract', 'achristmascarol', 'shrekthemusical', 'pajamagame', 'bonnieclyde', 'beetlejuice', 'tarzanoftheapes', 'anewbrain', 'tuckeverlasting', 'drseusshowthegrinchstolechristmas', 'ohcalcutta', 'dogfight', 'suessicalthemusical', 'twisted', 'shenandoah', 'coco', 'cyrano', 'ballroom', 'themagicshow', 'themagicshow', 'dancin', 'bigdeal']


In [10]:
# How many musicals were missed?
len(missed_musicals)

36

# Export Data

In [11]:
# Add synopses to dataframe
names['synopsis'] = synopses
names.head()

musical  \
0         lesmiserables   
1  thephantomoftheopera   
2              hamilton   
3         westsidestory   
4                wicked   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [12]:
# Export new dataframe
names.to_csv('./data/musical_synopses.csv', index=False)

In [ ]:
# Actually on it, figure it out
# A chorus line
# Gypsy
# Cinderella
# Billy elliott
# school of rock
# A little night music

...
# Tarzan


In [ ]:
res = requests.get('https://www.allmusicals.com/lyrics/secretgardenthe/synopsis.htm')

In [ ]:
# Set up error warnings
if res.status_code != 200:
    raise NameError('This is not a valid webpage. Try searching another link.')
        

In [ ]:
for name in names['musical']:
    if name[:3] == 'the':
        print(name)

In [ ]:
fnurl = baseurl1 + 'findingneverland' + baseurl2

In [ ]:
res = requests.get(baseurl1 + 'thephantomoftheoperathe' + baseurl2)

In [ ]:
if the title starts with the
try no the
if status_code != 200
try the at the end
try the at te beginning
try the at the beginning and end

if res.status_code != 200:
    try {
        
    }

In [ ]:
res = requests.get(fnurl)
res.status_code

In [ ]:
soup = BeautifulSoup(res.content)
print(soup.prettify()[:1000])

In [ ]:
fn_synopsis = soup.find('div', {'id': 'page'}).text.replace('\r', '')
fn_synopsis_list = [words.strip() for words in fn_synopsis.split('\n')[2:-2]]
fn_synop = ' '.join(fn_synopsis_list).strip()
fn_synop